## Install dependancies

In [ ]:
import cv2
import albumentations as A
import matplotlib.pyplot as plt
import shutil
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from PIL import Image
import os
import numpy as np

## Create Train and Val Set

In [ ]:
val_image_path = 'data/valid/images'
val_label_path = 'data/valid/labels'

os.makedirs(val_image_path, exist_ok = True)
os.makedirs(val_label_path, exist_ok = True)

# Val set: 7% of the total images
thresh = int(0.07 * len(os.listdir('data/images/')))
cnt =0 

#Loop through images in the dataset
for (root, dirs, files) in os.walk('data/images/'):
    if root.endswith('data') or root.endswith('Store'):
        continue
    else:
        np.random.shuffle(files)
        
        for image in files:
            if cnt == thresh:
                break
                
            if not image.endswith('Store'):
                shutil.copy(os.path.join(root, image),  val_image_path)
                os.remove(os.path.join(root, image))
                shutil.copy(os.path.join(root.split('/')[0], 'labels', image.replace('.jpg', '.txt')), val_label_path)
                os.remove(os.path.join(root.split('/')[0], 'labels', image.replace('.jpg', '.txt')))
                cnt += 1

## Define augmentation pipeline

In [ ]:
aug = A.Compose([A.OneOf([A.RandomBrightnessContrast(brightness_limit=[-0.15,0.30], contrast_limit=[-0.08,0.25], brightness_by_max=False,always_apply=True, p=0.9),
                                A.RandomToneCurve(scale=0.3, always_apply=True, p=0.9),
                                A.RGBShift(r_shift_limit=9, g_shift_limit=7, b_shift_limit=10, p=0.9),
                                A.ChannelDropout(p = 1.0),
                                A.ToGray(p=0.9),
                                A.ChannelShuffle(p = 0.9),
                                A.Blur(blur_limit =3, p = 0.8),
                                A.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.7)], p = 1.0)])

#### Augment each image twice and write to disk along with the labels

In [ ]:
cnt = 0

#Loop through the images of train set
for (root, dirs, files) in os.walk('final_dataset/train/images'):
    for img_path in files:
        
        if not img_path.endswith('Store'):
            image = cv2.imread(os.path.join(root, img_path))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #perform 2 augmentations for each image
            for i in range(2):
                aug_img = aug(image = image)['image']
                aug_name = f"{img_path.split('.')[0]}_aug{i}.jpg"
                
                #Write the augmented image to image folder
                cv2.imwrite(os.path.join(root, aug_name), aug_img)
                
                #define source and destination of yolo's label file
                label_source = os.path.join('/'.join(root.split('/')[:-1]) , 'labels', img_path.replace('.jpg', '.txt'))
                label_dest  = os.path.join('/'.join(root.split('/')[:-1]) , 'labels', aug_name.replace('.jpg', '.txt'))
                shutil.copyfile(label_source, label_dest)
                
        #prints after every 100 images         
        cnt += 1
        if cnt%100==0:
            print(f'{cnt} images done')

## Training

#### Define paths

In [2]:
project_path = "/content/drive/MyDrive/car_classification/"
yolo_path = project_path + 'yolov5/'

#### Clone yolo repo and install dependancies

In [ ]:
# Clone the yolov5 Repo
%cd {project_path}
!git clone https://github.com/ultralytics/yolov5.git

#Move the dataset inside yolo folder
!mv final_dataset.zip yolov5

%cd {yolo_path}
!pip install -r requirements.txt
!pip install wandb

#unzip the dataset
!unzip final_dataset.zip


#### Train yolov5 model
* Model used: ```yolov5m``` 
* Optimizer: ```SGD```

In [ ]:
%cd {yolo_path}
!python train.py --img 640 --batch 32 --epochs 100 --data ./final_dataset/data.yaml --weights yolov5m --cache  --name "yolov5medium"

#### Train yolov5 model 
* Model used: best weights from ```yolov5m``` training
* Freeze Backbone

In [4]:
%cd {yolo_path}

!python train.py --img 640 --batch 32 --epochs 30 --data ./final_dataset/data.yaml --weights runs/train/yolov5medium2/weights/best.pt --cache --freeze 10 --name "yolov5med_head_frozen"

/content/drive/MyDrive/car_classification/yolov5


#### Train yolov5 model
* Model used: ```yolov5s``` 

In [5]:
%cd {yolo_path}
!python train.py --img 640 --batch 32 --epochs 100 --data ./final_dataset/data.yaml --weights yolov5s.pt --cache --name "yolov5small"

wandb: Currently logged in as: sumitkutty (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=./final_dataset/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5small, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-197-g0cf932b torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, ob

#### Train yolov5 model
* Further train by using the weights of ```yolov5m```
* Optimizer: ```Adam```
* Epochs: ```50```

In [ ]:
!python train.py --img 640 --batch 64 --epochs 50 --data ./final_dataset/data.yaml --weights runs/train/yolov5medium2/weights/best.pt --optimizer 'Adam' --cache --name "yolov5medcontinued"

#### Train yolov5 model
* Further train by using the weights of ```yolov5medcontinued```
* Optimizer: ```Adam```
* Epochs: ```50```
* Freeze the network

**Achieved the best result**

In [ ]:
!python train.py --img 640 --batch 64 --epochs 50 --data ./final_dataset/data.yaml --weights runs/train/yolov5medcontinued2/weights/best.pt --optimizer 'Adam' --cache --name "yolov5medcontinued-froznet" --freeze 24

#### Perform Inference on val set
* Model used: best weight ```yolov5medcontinued-froznet``` 
* conf_thresh = 0.25

In [8]:
%cd {yolo_path}
!python detect.py --weights runs/train/yolov5medcontinued-froznet/weights/best.pt --img 640 --conf 0.3 --source ./final_dataset/valid/images

detect: weights=['runs/train/yolov5medium2/weights/best.pt'], source=./final_dataset/valid/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-197-g0cf932b torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

Fusing layers... 
Model Summary: 290 layers, 20856975 parameters, 0 gradients, 48.0 GFLOPs
image 1/156 /content/drive/MyDrive/car_classification/yolov5/final_dataset/valid/images/image_000000005.jpg: 608x640 10 persons, 1 car, Done. (0.020s)
image 2/156 /content/drive/MyDrive/car_classification/yolov5/final_dataset/valid/images/image_000000010.jpg: 448x640 1 person, Done. (0.048s)
image 3/156 /content/drive/MyDrive/car_classifica

## THE END